# note in this code we  have driver excel as casual which is 127 indicator and 14 sp 500 stock as effect how they are effected by world indicators

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
daily=pd.read_excel(r'/content/daily.xlsx')
data_driver=pd.read_excel(r'/content/data_drivers_2008_full_cutcolumns.xlsx')
sp500=pd.read_excel(r'/content/constituents_SPX_2008.xlsx')

In [ ]:
sp500.rename(columns={ "MCDONALD'S CORP": 'MCDONALDS CORP'}, inplace=True)

In [ ]:

daily = daily.fillna(method='ffill')

data_driver = data_driver.fillna(method='ffill')
data_driver['Date'] = pd.to_datetime(data_driver['Date'], format='%Y%m%d')
sp500 = sp500.fillna(method='ffill')
sp500['Date'] = pd.to_datetime(sp500['Date'], format='%Y%m%d')

In [ ]:
daily = daily.fillna(method='ffill')
columns_to_drop = ['Unnamed: 0', 'Datetime', ]
daily = daily.drop(columns=columns_to_drop)
daily.head()

In [ ]:
daily.set_index('Date', inplace=True)
data_driver.set_index('Date', inplace=True)
sp500.set_index('Date', inplace=True)

In [ ]:
sp500=sp500[['APPLE INC','GOLDMAN SACHS GROUP INC', 'MCDONALDS CORP','SCHLUMBERGER LTD']]

In [ ]:
print(data_driver.shape)
print(sp500.shape)


In [ ]:
# Assuming you have identified columns with negative values in data_driver and sp500
columns_with_negatives1 = data_driver.columns[data_driver.lt(0).any()]
columns_with_negatives2 = sp500.columns[sp500.lt(0).any()]
print(columns_with_negatives1)
print(columns_with_negatives2)
print(len(columns_with_negatives1))
print(len(columns_with_negatives2))

# Remove columns with negative values from data_driver
data_driver = data_driver.drop(columns=columns_with_negatives1)

# Remove columns with negative values from sp500
sp500 = sp500.drop(columns=columns_with_negatives2)




In [ ]:
# we only work on the data from 2014
df = pd.merge(data_driver, sp500, on='Date')
#df=df.tail(300)
print(len(df))
epsilon = 1e-8
df = df.replace([np.inf, -np.inf], np.nan).fillna(method='ffill')
df_after_2014 = df[df.index.year >= 2014]
print(len(df_after_2014))
df_after_2014=np.log(df_after_2014 ).diff().fillna(method='ffill')
df_after_2014=df_after_2014.dropna()
df_after_2014 = df_after_2014.drop(df_after_2014.index[-1])
df_after_2014

In [ ]:
plt.figure(figsize=(10, 6))
df_after_2014.plot()
plt.axhline(y=0.3, color='red', linestyle='--', label='Threshold Line')
crossing_legends = [] 
for column in df_after_2014.columns:
    crossing_points = df_after_2014[df_after_2014[column] > 0.3]
    if not crossing_points.empty:
        plt.scatter(crossing_points.index, crossing_points[column], marker='o', label=f'{column} Crossing')
        crossing_legends.append(f'{column} Crossing')
plt.xlabel('Date')
plt.ylabel('Log Difference')
plt.title('Stock Log Differences After 2014')
plt.legend(crossing_legends)
plt.show()


In [ ]:
df=df_after_2014.copy()

In [ ]:
df1 = pd.DataFrame()
for i in range(0,data_driver.shape[1]):
    causal = df.iloc[:, i]

    order_ar22 = (2, 0, 1)
    model_ar22 = ARIMA(causal, order=order_ar22,)
    results_ar22 = model_ar22.fit()
    causal_returns = results_ar22.fittedvalues

    # Append to df2 with a meaningful column name

    col_name = df.columns[i]
    df1[col_name] = causal_returns



In [ ]:
df2 = pd.DataFrame()
num_columns = len(df.columns)
for i in range(data_driver.shape[1], sp500.shape[1]+data_driver.shape[1]):
    if i < num_columns:
        effect = df.iloc[:, i]

        half_length_effect = len(effect) // 2
        first_half_effect = effect[:half_length_effect]
        second_half_effect = effect[half_length_effect:]

        order_ar11 = (2, 0, 1)
        order_arma12 = (2, 0, 2)

        model_ar11 = ARIMA(first_half_effect, order=order_ar11)
        results_ar11 = model_ar11.fit()

        model_arma12 = ARIMA(second_half_effect, order=order_arma12)
        results_arma12 = model_arma12.fit()

        effect_returns_arma = pd.concat([results_ar11.fittedvalues, results_arma12.fittedvalues])


        col_name = df.columns[i]
        df2[col_name] = effect_returns_arma
    else:
        print(f"Index {i} is out of bounds for the DataFrame.")


plt.plot(df2)


In [ ]:
plt.plot(df2)

In [ ]:
df = pd.merge(df1, df2, on='Date')
df = df.fillna(method='ffill')
df


In [ ]:
ce_df3=df.copy()
ce_df3=ce_df3.dropna()

In [ ]:
ce_df3.iloc[:,109]


In [ ]:
df1.index[half_length_effect]

In [ ]:
change_day_for_arma_index = ce_df3.index[half_length_effect]
k = 200
lag = 5
interval = 100

for f in range(data_driver.shape[1], data_driver.shape[1] +sp500.shape[1]):
    Data_res_all_columns = pd.DataFrame()
    for col_index in range(0, data_driver.shape[1]):
        Data_res = pd.DataFrame()
        for i in range(k, ce_df3.shape[0]):
            Data_k = ce_df3.iloc[i - interval:i, :]
            Data_k = (Data_k - Data_k.mean()) / Data_k.std()
            Date_ref = ce_df3.index[i]
            V_values = []
            for j in range(lag + 1):
                X = Data_k.iloc[:, col_index].values.reshape(-1, 1)
                Y = Data_k.iloc[:, f].values.reshape(-1, 1)
                Data_pca = pd.DataFrame(np.column_stack((X, Y)))

                Data_pca[1] = Data_pca[1].shift(periods=j)
                Data_pca = Data_pca.replace([np.inf, -np.inf], np.nan)
                Data_pca = Data_pca.fillna(0)

                pca = PCA(n_components=2)
                pca.fit(Data_pca)
                eigenvalues = pca.explained_variance_
                eigenvalue_ratio = eigenvalues[0] / (eigenvalues[0] + eigenvalues[1])
                V_values.append(eigenvalue_ratio)

            std_V_values = np.std(V_values)
            col_name = ce_df3.columns[col_index]
            Data_r = pd.DataFrame([[std_V_values]], index=[Date_ref], columns=[col_name])

            Data_res = pd.concat([Data_res, Data_r])

        Data_res_all_columns = pd.concat([Data_res_all_columns, Data_res], axis=1)


   
    Data_res_all_columns.to_excel(f"{ce_df3.columns[f]}_60.xlsx", index=True)
